##### Data preparation for insertion in the database

In [2]:
import pandas as pd
import numpy as np

files = {
    'name_basics': 'truncated_name.basics.tsv',
    'title_akas': 'truncated_title.akas.tsv',
    'title_basics': 'truncated_title.basics.tsv',
    'title_crew': 'truncated_title.crew.tsv',
    'title_episode': 'truncated_title.episode.tsv',
    'title_principals': 'truncated_title.principals.tsv',
    'title_ratings': 'truncated_title.ratings.tsv'
}

replacement_dict = {
    '\\N': None
}

dataframes = {key: pd.read_csv(files[key], sep = '\t', low_memory=False) for key in files}

Preparing to add data to the Title table:

In [3]:
import random

title_df = pd.merge(dataframes['title_basics'], dataframes['title_ratings'], how='left', on='tconst')
title_df = title_df[['tconst', 'primaryTitle', 'titleType', 'startYear', 'endYear', 'runtimeMinutes', 'averageRating', 'numVotes', 'isAdult', 'img_url_asset']]
title_df.columns = ['Title_ID', 'Original_Title', 'Type', 'Start_Year', 'End_Year', 'Runtime', 'Average_Rating', 'Votes', 'isAdult', 'IMAGE']
title_df.fillna(random.randint(1, 10), inplace=True)
title_df.replace('\\N', None, inplace=True)

Preparing for Alt_Title table:

In [4]:
alt_title_df = dataframes['title_akas'][['titleId', 'ordering', 'title', 'region']]
alt_title_df.columns = ['Title_FK', 'Ordering', 'Title_AKA', 'Region']
alt_title_df.fillna(random.randint(1, 100), inplace=True)
alt_title_df.replace('\\N', None, inplace=True)

/tmp/ipykernel_119507/3175029236.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alt_title_df.fillna(random.randint(1, 100), inplace=True)
/tmp/ipykernel_119507/3175029236.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alt_title_df.replace('\\N', None, inplace=True)


Episode table:

In [5]:
episode_df = dataframes['title_episode'][['tconst', 'parentTconst', 'seasonNumber', 'episodeNumber']]
episode_df.columns = ['Title_ID', 'parentTitle_ID', 'Season', 'Episode_Num']
episode_df.fillna(random.randint(1, 100), inplace= True)
episode_df.replace('\\N', None, inplace=True)


Person table:

In [6]:
person_df = dataframes['name_basics'][['nconst', 'primaryName', 'birthYear', 'deathYear', 'img_url_asset']]
person_df.columns = ['Name_ID', 'Name', 'Birth_Year', 'Death_Year', 'Image']
person_df.fillna(random.randint(1, 100), inplace=True)
person_df.replace('\\N', None, inplace=True)

/tmp/ipykernel_119507/1670186915.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  person_df.fillna(random.randint(1, 100), inplace=True)
/tmp/ipykernel_119507/1670186915.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  person_df.replace('\\N', None, inplace=True)


Participates_In Table:

In [7]:
dataframes['title_principals']['category'] = dataframes['title_principals']['category'].replace('\\N', '')
dataframes['title_principals']['job'] = dataframes['title_principals']['job'].replace('\\N', '')

dataframes['title_principals']['movie_job'] = dataframes['title_principals'].apply(lambda row: row['category'] + (',' + row['job'] if row['job'] != '' and row['job'] != row['category'] else ''), axis=1)

participates_in_df = dataframes['title_principals'][['tconst', 'nconst', 'ordering', 'movie_job', 'characters']]
participates_in_df.columns = ['Title_FK', 'Name_FK', 'Ordering', 'Job_Category', 'Character']
participates_in_df.fillna(random.randint(1, 100), inplace=True)
participates_in_df.replace('\\N', None, inplace=True)

/tmp/ipykernel_119507/3039884491.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  participates_in_df.fillna(random.randint(1, 100), inplace=True)
/tmp/ipykernel_119507/3039884491.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  participates_in_df.replace('\\N', None, inplace=True)


Genre and Title_Genre:

In [8]:
unique_genres = set()
for genre_list in dataframes['title_basics']['genres'].dropna().str.split(','):
    unique_genres.update([genre.strip() for genre in genre_list])

genres_df = pd.DataFrame({'Genre': list(unique_genres)})
genres_df.replace('\\N', None, inplace=True)
genres_df.dropna(how='all', inplace=True)

print(genres_df)

title_genre_df = dataframes['title_basics'][['tconst', 'genres']]
title_genre_df.columns = ['Title_FK', 'Genre']
title_genre_df['Genre'] = title_genre_df['Genre'].str.split(',')
title_genre_df = title_genre_df.explode('Genre')
title_genre_df.fillna(random.randint(1, 100), inplace=True)
title_genre_df.replace('\\N', None, inplace=True)

          Genre
1         Adult
2       Western
3     Adventure
4           War
5       Romance
6        Comedy
7        Action
8        Family
9       Mystery
10        Sport
11        Drama
12      History
13        Short
14  Documentary
15      Musical
16    Biography
17       Sci-Fi
18        Music
19       Horror
20         News
21        Crime
22      Fantasy
23    Animation
24     Thriller


/tmp/ipykernel_119507/3299006675.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title_genre_df['Genre'] = title_genre_df['Genre'].str.split(',')


Now, we push all the processed data into our database:

In [9]:
import re

# Create a list of items to iterate over
items_to_iterate = list(globals().items())

for var_name, value in items_to_iterate:
    if isinstance(value, pd.DataFrame) and re.search('[a-zA-Z]', var_name):
        print(f"DataFrame '{var_name}' columns:")
        print(value.columns.tolist())
        print()

DataFrame 'title_df' columns:
['Title_ID', 'Original_Title', 'Type', 'Start_Year', 'End_Year', 'Runtime', 'Average_Rating', 'Votes', 'isAdult', 'IMAGE']

DataFrame 'alt_title_df' columns:
['Title_FK', 'Ordering', 'Title_AKA', 'Region']

DataFrame 'episode_df' columns:
['Title_ID', 'parentTitle_ID', 'Season', 'Episode_Num']

DataFrame 'person_df' columns:
['Name_ID', 'Name', 'Birth_Year', 'Death_Year', 'Image']

DataFrame 'participates_in_df' columns:
['Title_FK', 'Name_FK', 'Ordering', 'Job_Category', 'Character']

DataFrame 'genres_df' columns:
['Genre']

DataFrame 'title_genre_df' columns:
['Title_FK', 'Genre']



Insert into Title:

In [10]:
import pymysql
import os
from dotenv import load_dotenv
import platform
from subprocess import call

load_dotenv()

host = os.environ.get('DB_HOST')
user = os.environ.get('DB_USER')
password = os.environ.get('DB_PASSWD')
db = os.environ.get('DB_NAME')
script_path = os.path.abspath('../ntuaflix_ddl.sql')

def run_ddl_script(host, user, password, script_path):
    if platform.system() == "Windows":
        command = f'mysql -h {host} -u {user} -p{password} < "{script_path}"'
        os.system(f'cmd /c "{command}"')
    else:
        os.system(f"mysql -h {host} -u {user} -p{password} < {script_path}")

run_ddl_script(host, user, password, script_path)

connection = pymysql.connect(host=host, user=user, password=password, db=db ,unix_socket='/Applications/XAMPP/xamppfiles/var/mysql/mysql.sock'
                             )
print("Connection to the database successful")

title_primary_keys = {}
with connection.cursor() as cursor:
    # Dealing with Titles first
    q = "INSERT INTO `Title` (Title_ID, Original_Title, Type, Start_Year, End_Year, Runtime, Average_Rating, Votes, isAdult, IMAGE) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
    for index, row in title_df.iterrows():
        tconst = row['Title_ID']
        original_title = row['Original_Title']
        type = row['Type']
        start_year = row['Start_Year']
        end_year = row['End_Year']
        runtime = row['Runtime']
        average_rating = row['Average_Rating']
        votes = row['Votes']
        is_adult = row['isAdult']
        image = row['IMAGE']

        cursor.execute(q, (tconst, original_title, type, start_year, end_year, runtime, average_rating, votes, is_adult, image))
        connection.commit()
        print("cursor commited")
    
        # Saving the primary key
        title_primary_keys[tconst] = cursor.lastrowid

Connection to the database successful
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cursor commited
cu

Insert into Person:

In [11]:
person_primary_keys = {}

with connection.cursor() as cursor:
    query = "INSERT INTO `Person` (Name_ID, Name, Image, Birth_Year, Death_Year) VALUES (%s, %s, %s, %s, %s)"
    
    for index, row in person_df.iterrows():
        name_id = row['Name_ID']
        name = row['Name']
        image = row['Image']
        birth_year = row['Birth_Year']
        death_year = row['Death_Year']

        try:
            cursor.execute(query, (name_id, name, image, birth_year, death_year))
            connection.commit()
        
            # Saving the auto-generated primary key
            person_primary_keys[name_id] = cursor.lastrowid
        except Exception as e:
            connection.rollback()  # Rollback in case of error


Insert into Participates_In:

In [12]:
with connection.cursor() as cursor:
    query = "INSERT INTO `Participates_In` (Title_FK, Name_FK, Ordering, Job_Category, `Character`) VALUES (%s, %s, %s, %s, %s)"
    
    for index, row in participates_in_df.iterrows():
        title_fk = title_primary_keys.get(row['Title_FK'])
        name_fk = person_primary_keys.get(row['Name_FK'])
        ordering = row['Ordering']
        job_category = row['Job_Category']
        character = row['Character']

        if title_fk is not None and name_fk is not None:
            cursor.execute(query, (title_fk, name_fk, ordering, job_category, character))
            connection.commit()
        else:
            print("an error occured")


Genre and Title_Genre:

In [13]:
with connection.cursor() as cursor:
    genre_query = "INSERT INTO `Genre` (Genre) VALUES (%s)"
    genre_primary_keys = {}

    print(genres_df)
    for index, row in genres_df.iterrows():
        genre = row['Genre']
        cursor.execute(genre_query, (genre,))
        connection.commit()

        # Saving the primary key
        genre_primary_keys[genre] = cursor.lastrowid

    title_genre_query = "INSERT INTO `Title_Genre` (Title_FK, Genre_FK) VALUES (%s, %s)"

    for index, row in title_genre_df.iterrows():
        title_fk = title_primary_keys.get(row['Title_FK'])
        if title_fk == None:
            continue
        genre_fk = genre_primary_keys.get(row['Genre'])


        try:
            if title_fk is not None and genre_fk is not None:
                cursor.execute(title_genre_query, (title_fk, genre_fk))
                connection.commit()
            else:
                connection.rollback()
        except Exception as e:
            print(str(e))

          Genre
1         Adult
2       Western
3     Adventure
4           War
5       Romance
6        Comedy
7        Action
8        Family
9       Mystery
10        Sport
11        Drama
12      History
13        Short
14  Documentary
15      Musical
16    Biography
17       Sci-Fi
18        Music
19       Horror
20         News
21        Crime
22      Fantasy
23    Animation
24     Thriller


Episodes

In [14]:
episode_query = "INSERT INTO `Episodes` (Title_FK, Parent_Title_FK, Season, Episode_Num) VALUES (%s, %s, %s, %s)"

with connection.cursor() as cursor:
    for index, row in episode_df.iterrows():
        title_fk = title_primary_keys.get(row['Title_ID'])
        parent_title_fk = title_primary_keys.get(row['parentTitle_ID'])
        print(parent_title_fk)
        season = row['Season']
        episode_num = row['Episode_Num']

        # Ensure both foreign keys are found
        if title_fk is not None and parent_title_fk is not None:
            try:
                cursor.execute(episode_query, (title_fk, parent_title_fk, season, episode_num))
                connection.commit()
            except Exception as e:
                print(f"Error inserting episode: {e}")
                connection.rollback()


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


Alternative Title

In [15]:
alt_title_primary_keys = {}
count = 0

with connection.cursor() as cursor:
    query = "INSERT INTO `Alt_Title` (Title_FK, Ordering, Title_AKA, Region) VALUES (%s, %s, %s, %s)"
    
    for index, row in alt_title_df.iterrows():
        title_fk = title_primary_keys.get(row['Title_FK'])
        ordering = row['Ordering']
        title_aka = row['Title_AKA']
        region = row['Region']
        count += 1

        try:
            cursor.execute(query, (title_fk, ordering, title_aka, region))
            connection.commit()
        
            # Saving the auto-generated primary key
            alt_title_primary_keys[(title_fk, title_aka)] = cursor.lastrowid
        except Exception as e:
            print(f"Error inserting Title_FK: {title_fk}, Title_AKA: {title_aka}: {e}")
            connection.rollback()  # Rollback in case of error
